Steps 
- 

In [25]:
import cv2
import numpy as np

In [114]:
widthImage = 480
heightImage = 640

In [134]:

def stackImages(scale,imgArray):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                if imgArray[x][y].shape[:2] == imgArray[0][0].shape [:2]:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                else:
                    imgArray[x][y] = cv2.resize(imgArray[x][y], (imgArray[0][0].shape[1], imgArray[0][0].shape[0]), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv2.cvtColor( imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
        ver = np.vstack(hor)
    else:
        for x in range(0, rows):
            if imgArray[x].shape[:2] == imgArray[0].shape[:2]:
                imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            else:
                imgArray[x] = cv2.resize(imgArray[x], (imgArray[0].shape[1], imgArray[0].shape[0]), None,scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        ver = hor
    return ver

### Preprocessing function

In [115]:
kernel = np.ones((5,5))

def preProcessing(img):
    imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    imgBlur = cv2.GaussianBlur(imgGray,(5,5),sigmaX=1)
    imgCanny = cv2.Canny(imgBlur,200,200)
    imgDial = cv2.dilate(imgCanny,kernel,iterations=2)
    imgErode = cv2.erode(imgDial,kernel,iterations=1)

    return imgErode


### Contour function

- We aim to retrieve the biggest contour in our image

In [155]:
def getContours(img):
    maxArea = 0
    biggest = np.array([])
    contours, hierarchy = cv2.findContours(img,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 1000:
            #cv2.drawContours(imgContour,cnt,-1,(255,0,0),20)
            peri = cv2.arcLength(cnt,closed=True)
            approx = cv2.approxPolyDP(cnt,0.02*peri,closed=True)
            if area > maxArea and len(approx) == 4:
                biggest = approx
                maxArea = area
    cv2.drawContours(imgContour,biggest,-1,(255,0,0),20)
    return biggest

### Warping the biggest contour 

- Now that we have our biggest contour , we need to warp it and get its' birds eye view
- The main thing we need to contend with is the order in which the getPerspectieTransform() expects the points. If the points obtained via biggest don't follow the format , it might lead to unexpected / unwanted results
- We reorder the given coordinates and then use it to in getPerspectiveTransform() and warpPerspective()

In [178]:
def getWarp(img,biggest):
    biggest = reorder(biggest)
    pts1 =  np.float32(biggest)
    pts2 = np.float32([[0,0],[widthImage,0], [0,heightImage],[widthImage,heightImage]])
    matrix = cv2.getPerspectiveTransform(pts1,pts2)
    imgOutput = cv2.warpPerspective(img,matrix,(widthImage,heightImage))
    
    imgCropped  = imgOutput[20:imgOutput.shape[0]-20,20:imgOutput.shape[1]-20]
    imgCropped = cv2.resize(imgCropped,(widthImage,heightImage))

    return imgCropped

### Reorder function

- The following function will take the coordinates of the biggest contour and rearrange them in the expected order
- The way we do this is by adding the coordinates of the corners.(rowwise addition)
- The one with the least sum would be the origin while the one with the highest would be the extreme corner(diagonal opposite of origin)
- Take rowwise difference .Least difference second point, max difference, third point

In [179]:
def reorder(myPoints):
    myPoints = myPoints.reshape((4,2))
    myPointsNew = np.zeros((4,1,2),np.int32)
    add = myPoints.sum(1)
    #print('add',add)

    myPointsNew[0] = myPoints[np.argmin(add)]
    myPointsNew[3] = myPoints[np.argmax(add)]


    diff = np.diff(myPoints,axis=1)
    myPointsNew[1] = myPoints[np.argmin(diff)]
    myPointsNew[2] = myPoints[np.argmax(diff)]

    #print('New Points ',myPointsNew)

    return myPointsNew

For test image

In [180]:
img = cv2.imread('D:\\Courses\\OpenCV\\Resources\\document.jpg')
img = cv2.resize(img,(widthImage,heightImage))
imgContour = img.copy()
imgThres = preProcessing(img)

biggest = getContours(imgThres)

if biggest.size != 0:
    imgWarped = getWarp(img,biggest)
    imgArray = [[img,imgContour],
            [imgThres,imgWarped]]

else:
    imgArray = [[img,imgContour],
            [imgThres,img]]


stackedImages = stackImages(0.6,imgArray)
cv2.imshow('Stacked Images',stackedImages)
cv2.waitKey(0)


-1

In [182]:
'''
cap = cv2.VideoCapture(0)
cap.set(3,widthImage)
cap.set(4,heightImage)

while True:
    success, img = cap.read()
    imgContour = img.copy()
    img = cv2.resize(img,(widthImage,heightImage))
    
    imgThres = preProcessing(img)
    biggest = getContours(imgThres)
    
    if biggest.size != 0:
        imgWarped = getWarp(img,biggest)
        imgArray = [[img,imgContour],
                    [imgThres],imgWarped]
    else:
        imgWarped = getWarp(img,biggest)
        imgArray = [[img,imgContour],
                    [imgThres],img]

    stackedImages = stackImages(0.6,imgArray)

    cv2.imshow('Video',stackedImages)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
cap.release()
'''

"\ncap = cv2.VideoCapture(0)\ncap.set(3,widthImage)\ncap.set(4,heightImage)\n\nwhile True:\n    success, img = cap.read()\n    imgContour = img.copy()\n    img = cv2.resize(img,(widthImage,heightImage))\n    \n    imgThres = preProcessing(img)\n    biggest = getContours(imgThres)\n    \n    if biggest.size != 0:\n        imgWarped = getWarp(img,biggest)\n        imgArray = [[img,imgContour],\n                    [imgThres],imgWarped]\n    else:\n        imgWarped = getWarp(img,biggest)\n        imgArray = [[img,imgContour],\n                    [imgThres],img]\n\n    stackedImages = stackImages(0.6,imgArray)\n\n    cv2.imshow('Video',stackedImages)\n\n    if cv2.waitKey(1) & 0xFF == ord('q'):\n        break\n\ncv2.destroyAllWindows()\ncap.release()\n"

In [183]:
cap.release()

In [175]:
cv2.destroyAllWindows()